In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004174232482910156
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 12.466964424282834
200000 19.2469264268968
300000 18.793666958314404
400000 18.717859791861688
500000 18.78896094165359
600000 19.419026682022714
700000 18.599864078101824
800000 18.568981920641814
900000 18.731016343097856
1000000 16.35859802117336
1100000 8.671311273763154
1200000 18.515479940855986
1300000 18.36809484135063
1400000 18.352465952016612
1500000 18.29714130312771
1600000 19.338608488545074
1700000 18.394635561004243
1800000 18.2409183241833
1900000 18.869999087829903
2000000 18.88802080508979
2100000 4.950068445313608
2200000 20.92801281267294
2300000 22.241938402783006
2400000 21.897361155978405
2500000 21.802072728074883
2600000 22.591781405757995
2700000 21.063889339955203
2800000 20.845471999318036
2900000 20.802769219277227
3000000 20.

29400000 16.66797628606468
29500000 16.749049847923935
29600000 15.993500520981664
29700000 11.442205955037227
29800000 11.196577808391881
29900000 16.041473284375357
30000000 16.50778413145346
30100000 16.38326785514258
30200000 17.737589287164287
30300000 17.32181031866189
30400000 18.072524838267675
30500000 11.736725710951156
30600000 14.174324219719887
30700000 17.87996141126803
30800000 17.607622017435993
30900000 17.52225080121751
31000000 9.35917065176729
31100000 15.523874446812039
31200000 17.579591272432214
31300000 17.237273884020862
31400000 17.306605875303646
31500000 16.27352089722367
31600000 15.885495858810788
31700000 16.07265520565794
31800000 5.326859008217426
31900000 16.37468784597147
32000000 15.620923905516213
32100000 15.962249597026906
32200000 16.313936679794306
32300000 17.387878719045816
32400000 17.357254796653816
32500000 15.384099567220316
32600000 10.31750215244974
32700000 17.410101264113038
32800000 17.32685831377888
32900000 17.609271326800613
330000

59100000 15.096272026743339
59200000 15.743657046677798
59300000 15.468982336984709
59400000 15.701237637599185
59500000 15.882215697976719
59600000 15.188571369557376
59700000 15.425599887392906
59800000 14.808439136070952
59900000 8.43289574341985
60000000 12.434964885727833
60100000 13.455219924517518
60200000 13.927218005799833
60300000 13.335361520827094
60400000 12.570762981511555
60500000 6.631226565290558
60600000 13.398718248706212
60700000 13.558508323126038
60800000 13.159097294814583
60900000 13.234972964062342
61000000 8.181062197222271
61100000 13.550070312083124
61200000 14.984250163824665
61300000 14.68022500088881
61400000 15.47786220044092
61500000 15.877607941364074
61600000 15.464314835768986
61700000 14.888593614204789
61800000 15.062602831768809
61900000 15.114500102453064
62000000 6.240333939501032
62100000 13.617122033195871
62200000 13.430288239277639
62300000 13.297843734661711
62400000 13.435205385624956
62500000 6.639147481437611
62600000 12.56452510274412
6

88700000 11.26302416010261
88800000 7.076383871346692
88900000 10.521812074382426
89000000 11.93053489341425
89100000 8.910120226941636
89200000 7.773217469860055
89300000 11.282281005575133
89400000 11.638266653366342
89500000 6.800596628233627
89600000 11.381948918504184
89700000 12.370337171936127
89800000 11.593717354411917
89900000 12.922776813081937
90000000 12.947197970800943
90100000 12.43157543709347
90200000 13.535650637883418
90300000 12.783878262902647
90400000 13.466804217679773
90500000 12.225082959689516
90600000 12.663666411598818
90700000 11.344752895620715
90800000 11.66241311331165
90900000 5.563343545718157
91000000 11.366030458321479
91100000 11.825276535286696
91200000 4.408743667608629
91300000 11.585739640873658
91400000 11.613841407539875
91500000 6.278158319217193
91600000 12.023710458002592
91700000 12.038008373329495
91800000 11.470701962110303
91900000 12.099697089343266
92000000 13.217395681169416
92100000 12.514442380127486
92200000 13.223740558522055
923

117800000 3.1541563691610697
117900000 4.100316329046188
118000000 9.363264106591156
118100000 9.260746233667483
118200000 10.088159786116815
118300000 9.333352986702199
118400000 9.372132654437875
118500000 10.059963861366855
118600000 9.668400205634455
118700000 10.63746870632913
118800000 10.319076838077615
118900000 11.361661578845833
119000000 9.460920578083924
119100000 9.473131823181074
119200000 9.98325965327405
119300000 9.28343377881109
119400000 10.092722127715808
119500000 9.31103110393627
119600000 8.838528895178749
119700000 10.166031191928042
119800000 3.103423399387549
119900000 1.7231837098600864
120000000 6.627375472691942
120100000 9.751284508883865
120200000 8.956294351246923
120300000 9.459936136233987
120400000 9.791066366747964
120500000 9.265643306304622
120600000 9.833059982029898
120700000 9.639497993402943
120800000 10.043525444949617
120900000 11.408013150796842
121000000 10.841754196403006
121100000 10.640916282643218
121200000 9.844847768278765
121300000 1

146600000 6.058659613756847
146700000 6.105136511145555
146800000 8.031144946734782
146900000 7.661278620877387
147000000 7.9634762975331865
147100000 8.255508276893323
147200000 7.743217450608176
147300000 8.431723801428408
147400000 8.422942399238977
147500000 9.128396657527848
147600000 8.666399465738367
147700000 8.684663667562706
147800000 7.707818137538533
147900000 7.761965832256315
148000000 8.515132647463767
148100000 7.569925677297151
148200000 8.188448448413165
148300000 7.317655175300269
148400000 3.640934549301368
148500000 6.683476535219639
148600000 5.163224604085463
148700000 3.3060309205705916
148800000 7.557064172173586
148900000 8.151727089908542
149000000 7.6675455731153015
149100000 8.388983825770513
149200000 7.639071880147717
149300000 8.268481957019944
149400000 8.032659722984839
149500000 8.558966677631592
149600000 8.940953360620888
149700000 8.344942950678275
149800000 8.164171966597568
149900000 7.6912853791086855
150000000 8.213669132834715
150100000 7.4003

175900000 6.60582587205937
176000000 6.302920137274212
176100000 6.827782013654152
176200000 6.335662442552035
176300000 6.767856867001954
176400000 6.285884761412807
176500000 6.605151024369002
176600000 5.977361109380249
176700000 5.903969727019344
176800000 4.842851475875895
176900000 3.8278530675468745
177000000 6.0881757304509225
177100000 5.908529835957349
177200000 3.142275682020744
177300000 5.296473538780205
177400000 5.6948694777631435
177500000 5.609554061878849
177600000 2.9277654627861747
177700000 5.763596498949692
177800000 5.644384777368265
177900000 6.690387069539751
178000000 6.090413703574478
178100000 6.684611469431789
178200000 6.3456359801141025
178300000 6.672164415680244
178400000 6.067717376286371
178500000 6.417930613976387
178600000 6.071623241667321
178700000 5.65228211184299
178800000 5.877976485270653
178900000 2.6542445531111736
179000000 5.614745863759214
179100000 5.957276021884883
179200000 5.307711106848892
179300000 2.3953380624397456
179400000 5.782

205100000 4.389441767668901
205200000 4.355890506636726
205300000 2.047886936830323
205400000 4.117430987559163
205500000 4.04140100031633
205600000 3.9525749875618996
205700000 3.864717623556244
205800000 3.8180032013244554
205900000 1.394908965501416
206000000 4.010374303905123
206100000 3.881650826263661
206200000 3.791867120216994
206300000 3.8875837867494267
206400000 4.101625189821526
206500000 1.7283641216514236
206600000 4.3300194955027695
206700000 4.305098282367041
206800000 4.251163664954357
206900000 4.255900513913785
207000000 4.316238179821548
207100000 4.227588112805892
207200000 4.222224165323554
207300000 2.8902149978579783
207400000 3.0645192027515558
207500000 3.9521788016429245
207600000 3.8007211467638387
207700000 3.585474690104349
207800000 3.6609415498332125
207900000 2.8705353673168337
208000000 2.3268951981875605
208100000 3.6694821593354883
208200000 3.6346954385214825
208300000 3.7853710183461273
208400000 3.881328542556286
208500000 3.637515128767309
208600

233800000 1.3721899853938633
233900000 2.162619343674336
234000000 2.202204656000936
234100000 2.0755575752127977
234200000 2.1292199196736394
234300000 2.053558467477356
234400000 1.9778285865935814
234500000 1.9938249119989397
234600000 0.6936779614935411
234700000 1.9343416404301006
234800000 1.9839709365142602
234900000 2.033535907829468
235000000 2.061121504803032
235100000 2.060938200446689
235200000 2.1068162656449867
235300000 2.078994106725046
235400000 0.8934112091987728
235500000 2.082977476058636
235600000 2.09931518601461
235700000 2.061845256148823
235800000 0.9045097532116201
235900000 2.024200048920315
236000000 2.0005444943627975
236100000 1.931551197595599
236200000 1.8529538342596792
236300000 1.9426841865550517
236400000 1.8107618107065084
236500000 1.7758251235614917
236600000 1.0280226758843463
236700000 1.4639512418511946
236800000 1.7669320519098572
236900000 1.8136577316581528
237000000 1.935168638995834
237100000 1.8528671719442844
237200000 1.8778571811138036

261800000 0.0025646796510775884
261900000 0.0019669938879516386
262000000 0.001339744429861837
262100000 0.0007081551831761996
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 5.767577387094498e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
